In [2]:
import requests
from bs4 import BeautifulSoup
import re
from weasyprint import HTML

In [3]:
url ="https://www.sentenzeappalti.it/sentenze/"
# Download the content directly
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

In [4]:
def sanitize_filename(name):    
    # Sostituiamo spazi con underscore
    name = name.replace(" ", "_")
    # Sostituiamo caratteri problematici con '-'
    name = re.sub(r'[\/,:]', '-', name)
    # Rimuoviamo tutti i caratteri non alfanumerici, tranne '-' e '_'
    name = re.sub(r'[^a-zA-Z0-9_-]', '', name)
    # Rimuoviamo punti interni nel nome del file (ma non nell'estensione)
    name = name.replace('.', '')

    return name

In [5]:
# Trova il div con class="pagination loop-pagination"
pagination_div = soup.find('div', class_='pagination loop-pagination')
maxpages = 0
# Estrai il valore numerico presente in p.text
for p in pagination_div.find_all("a", class_="page-numbers"):
    numeric_value = re.findall(r'\d+', p.text)
    if numeric_value:
        pages = int(numeric_value[0])
        if pages > maxpages:
            maxpages = pages

In [22]:
for npage in range(14,15): # range(maxpages):
    num_page = npage + 1
    print(num_page)
    url_page = url + "page"+ str(num_page)
    try:
        # Esegui la richiesta HTTP
        response = requests.get(url_page)
        response.raise_for_status()
        soup_page = BeautifulSoup(response.text, 'html.parser')
        headers = soup_page.find_all("header",class_="entry-header")
        for header in headers:
            url_sentence = header.find("h1").find("a")['href']
            try:
                response = requests.get(url_sentence)
                response.raise_for_status()
                soup_sentence = BeautifulSoup(response.text, 'html.parser')
                portali_giustizia_links_with_text = [(a['href'], a.text) for a in soup_sentence.find_all('a', href=True) if 'portali.giustizia' in a['href']]
                for portale in portali_giustizia_links_with_text:
                    url_to_pdf = portale[0]
                    name_pdf = sanitize_filename(portale[1]) + ".pdf"
                    try:
                        # Estrai l'URL dal primo elemento di portali_giustizia_links
                        # Leggi il contenuto dell'URL
                        response = requests.get(url_to_pdf)
                        response.raise_for_status()
                        content = response.text
                        HTML(string=content, base_url=url_to_pdf).write_pdf(name_pdf)
                    except requests.exceptions.RequestException as e:
                        print(f"Errore nella richiesta {url_to_pdf}: {e}")
                        continue
            except requests.exceptions.RequestException as e:
                print(f"Errore nella richiesta {url_sentence}: {e}")
                continue
    except requests.exceptions.RequestException as e:
        print(f"Errore nella richiesta {url_page}: {e}")
        continue

15


FileNotFoundError: [Errno 2] No such file or directory: 'art. 16 comma 5 All. II.12 D.lgs. n. 36/2023.pdf'